<center><img src="images/logo.png" alt="drawing" width="400" style="background-color:white; padding:1em;" /></center> <br/>

<div class="alert alert-block alert-success"><span style="color:blue"><h1>ML through Application</h1></span></div>
<div class="alert alert-block alert-warning">
<span style="color:blue"><h2>Module 2, Lab 6: Using Ensemble Learning</h2></span></div>

Ensemble methods create a strong model by combining the predictions of multiple weak models (also known as weak learners or base estimators) that are built with a given dataset and a given learning algorithm.

You will learn how to do the following:

- Explain what bagging is and how to use it.
- Explain what a random forest is and how to use it.
- Explain what boosting is and how to use it.

----

__Austin Animal Center Dataset__

In this lab, you will work with historical pet adoption data in the [Austin Animal Center Shelter Intakes and Outcomes dataset](https://www.kaggle.com/datasets/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes?resource=download). The target field of the dataset (**Outcome Type**) is the outcome of adoption: 1 for adopted and 0 for not adopted. Multiple features are used in the dataset.

Dataset schema:
- __Pet ID:__ Unique ID of the pet
- __Outcome Type:__ State of pet at the time of recording the outcome (0 = not placed, 1 = placed). This is the field to predict.
- __Sex upon Outcome:__ Sex of pet at outcome
- __Name:__ Name of pet 
- __Found Location:__ Found location of pet before it entered the shelter
- __Intake Type:__ Circumstances that brought the pet to the shelter
- __Intake Condition:__ Health condition of the pet when it entered the shelter
- __Pet Type:__ Type of pet
- __Sex upon Intake:__ Sex of pet when it entered the shelter
- __Breed:__ Breed of pet 
- __Color:__ Color of pet 
- __Age upon Intake Days:__ Age (days) of pet when it entered the shelter
- __Age upon Outcome Days:__ Age (days) of pet at outcome

----

You will be presented with a challenge in this notebook.<br/>

| <img style="float: center;" src="./images/challenge.png" alt="Challenge" width="125"/>|
| --- |
|<p style="text-align:center;">Challenges are where you can practice your coding skills.</p>

---
<div class="alert alert-block alert-success">
<h1><span style="color:blue">Index</span></h1></div>

- [Data preparation](#Data-preparation)
- [Bagging](#Bagging)
- [Random forest](#Random-forest)
- [Boosting](#Boosting)

---
<div class="alert alert-block alert-success">
<h1><span style="color:blue"> Data preparation</span></h1></div>

Before you can build the first model, you need to import and prepare the data. You can follow the same steps as in the previous labs.

In [1]:
%%capture
# Install libraries
!pip install -U -q -r requirements.txt

In [2]:
import re, string
import matplotlib.pyplot as plt

%matplotlib inline
import nltk
from nltk.stem import SnowballStemmer
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    BaggingClassifier,
    RandomForestClassifier,
    GradientBoostingClassifier,
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
df = pd.read_csv("data/review_dataset.csv")

print("The shape of the dataset is:", df.shape)

The shape of the dataset is: (95485, 13)


In [4]:
# Create lists of the features and name the target

# Numerical features 
numerical_features = ["Age upon Intake Days", "Age upon Outcome Days"]

# Drop the ID features: RescuerID and PetID
categorical_features = [
    "Sex upon Outcome",
    "Intake Type",
    "Intake Condition",
    "Pet Type",
    "Sex upon Intake",
    "Breed",
    "Color",
]

# Based on exploratory data analysis (EDA), select the text features
text_features = ["Found Location"]

model_features = numerical_features + categorical_features + text_features
model_target = "Outcome Type"

df[categorical_features + text_features] = df[
    categorical_features + text_features
].astype("str")

---
<div class="alert alert-block alert-warning">
<h3><span style="color:blue">Clean numerical features</span></h3></div>

__Note:__ The cleaning stage can take a few minutes, depending on how much text needs to be processed.

In [5]:
# Prepare cleaning functions

stop_words = ["a", "an", "the", "this", "that", "is", "it", "to", "and"]

stemmer = SnowballStemmer("english")


def preProcessText(text):
    # Lowercase text, and strip leading and trailing white space
    text = text.lower().strip()

    # Remove HTML tags
    text = re.compile("<.*?>").sub("", text)

    # Remove punctuation
    text = re.compile("[%s]" % re.escape(string.punctuation)).sub(" ", text)

    # Remove extra white space
    text = re.sub("\s+", " ", text)

    return text


def lexiconProcess(text, stop_words, stemmer):
    filtered_sentence = []
    words = text.split(" ")
    for w in words:
        if w not in stop_words:
            filtered_sentence.append(stemmer.stem(w))
    text = " ".join(filtered_sentence)

    return text


def cleanSentence(text, stop_words, stemmer):
    return lexiconProcess(preProcessText(text), stop_words, stemmer)


# Clean the text features
for c in text_features:
    print("Text cleaning: ", c)
    df[c] = [cleanSentence(item, stop_words, stemmer) for item in df[c].values]

Text cleaning:  Found Location


---
<div class="alert alert-block alert-warning">
<h3><span style="color:blue">Create training and test datasets</span></h3></div>


As part of data preparation, the dataset is split into training and test subsets by using sklearn's [train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.

For this notebook, you will use 90 percent of the data for the training set and 10 percent for the test set. Determine the best split based on the size of your dataset.

In [6]:
train_data, test_data = train_test_split(
    df, test_size=0.1, shuffle=True, random_state=23
)

The data is now prepared, and you are ready to create a bagging classifier.

---
<div class="alert alert-block alert-success">
<h1><span style="color:blue">Bagging</span></h1></div>


   (**Outcome Type**) is the outcome of adoption:<span style="color:red"><b> 1 for adopted and 0 for not adopted.</b></span>

<center><img src="Hyperparameter_Optimization_using_Grid_Search.svg.png" alt="drawing" width="800" style="background-color:white; padding:1em;" /></center>

In this section, you will build your first ensemble model by using the bootstrap aggregating, or bagging, approach. With this approach, you randomly draw multiple data subsets from the training set (with replacement) and train one model for each subset.

The first approach will use multiple trees in the bagging model.

---
<div class="alert alert-block alert-warning">
<h3><span style="color:blue">Data processing with a pipeline and a bagging ColumnTransformer</span></h3></div>


You need to use different pipelines to handle the numerical, categorical, and text features. Then, you will combine them into a composite pipeline along with an estimator. To do this, you will use a [BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html).

In [7]:
### COLUMN_TRANSFORMER ###
##########################

# Preprocess the numerical features
numerical_processor = Pipeline(
    [
        (
            "num_scaler",
            MinMaxScaler(),
        )  # Shown in case it is needed. Not a must with decision trees.
    ]
)

# Preprocess the categorical features
# handle_unknown tells it to ignore (rather than throw an error for) any value
# that was not present in the initial training set.
categorical_processor = Pipeline(
    [("cat_encoder", OneHotEncoder(handle_unknown="ignore"))]
)

# Preprocess the text feature
text_processor_0 = Pipeline(
    [("text_vect_0", CountVectorizer(binary=True, max_features=150))]
)

# Combine all data preprocessors from above (add more if you choose to define more)
# For each processor/step, specify a name, the actual process, and the features to be processed
data_preprocessor = ColumnTransformer(
    [
        ("numerical_pre", numerical_processor, numerical_features),
        ("categorical_pre", categorical_processor, categorical_features),
        ("text_pre_0", text_processor_0, text_features[0]),
    ]
)

### PIPELINE ###
################

# Pipeline with all desired data transformers, along with an estimator
# Later, you can set/reach the parameters by using the names issued - for hyperparameter tuning, for example

#####################################################
### Notice the pipeline using a BaggingClassifier ###
#####################################################
pipeline = Pipeline(
    [
        ("data_preprocessing", data_preprocessor),
        (
            "bg",
            BaggingClassifier(
                DecisionTreeClassifier(max_depth=25),  # Each tree has max_depth=25
                n_estimators=10,
            ),
        ),
    ]
)  # Use 10 trees

# Visualize the pipeline
# This will be helpful especially when building more complex pipelines,
# stringing together multiple preprocessing steps
from sklearn import set_config

set_config(display="diagram")
pipeline

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numerical_pre',
                                                  Pipeline(steps=[('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age upon Intake Days',
                                                   'Age upon Outcome Days']),
                                                 ('categorical_pre',
                                                  Pipeline(steps=[('cat_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex upon Outcome',
                                                   'Intake Type',
                                                   'Intake Condition',
                                                   'Pet Type',
                                                   'Sex upon Intake', 'Breed',
                                                   'Color']),
                                                 ('text_pre_0',
                                                  Pipeline(steps=[('text_vect_0',
                                                                   CountVectorizer(binary=True,
                                                                                   max_features=150))]),
                                                  'Found Location')])),
                ('bg',
                 BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=25)))])

Now you can fit the bagging model, and see the training and test scores.

In [8]:
# Get training data to train the pipeline
X_train = train_data[model_features]
y_train = train_data[model_target]

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Use the fitted pipeline to make predictions on the training dataset
train_predictions = pipeline.predict(X_train)
print(confusion_matrix(y_train, train_predictions))
print(classification_report(y_train, train_predictions))
print("Accuracy (training):", accuracy_score(y_train, train_predictions))

# Get testing data to test the pipeline
X_test = test_data[model_features]
y_test = test_data[model_target]

# Use the fitted pipeline to make predictions on the testing dataset
test_predictions = pipeline.predict(X_test)
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))
print("Accuracy (test):", accuracy_score(y_test, test_predictions))

[[29716  7783]
 [ 1929 46508]]
              precision    recall  f1-score   support

         0.0       0.94      0.79      0.86     37499
         1.0       0.86      0.96      0.91     48437

    accuracy                           0.89     85936
   macro avg       0.90      0.88      0.88     85936
weighted avg       0.89      0.89      0.89     85936

Accuracy (training): 0.8869856637497673
[[3046 1086]
 [ 445 4972]]
              precision    recall  f1-score   support

         0.0       0.87      0.74      0.80      4132
         1.0       0.82      0.92      0.87      5417

    accuracy                           0.84      9549
   macro avg       0.85      0.83      0.83      9549
weighted avg       0.84      0.84      0.84      9549

Accuracy (test): 0.8396690752958424


<center><img src="Confusion Matrix.png" alt="drawing" width="500" style="background-color:white; padding:1em;" /></center>


Using a bagging classifier isn't difficult because it only requires updating one line of code.

Next, you will create a random forest model.

---
<div class="alert alert-block alert-success">
<h1><span style="color:blue">Random forest</span></h1></div>


Now, you will try the second ensemble model: random forest. Random forest involves a similar ensemble process:
- Draw random subsets (with replacement) from the original dataset.
- Train individual trees with each subset.

However, a difference is that random forest uses a randomly selected feature subset for each tree. As a rule of thumb, pick the `sqrt(# features)` as the number of random features for each tree and don't use any other features.


The model is called in a similar way to the bagging method. You will replace the BaggingClassifier with a [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) in the pipeline.

In [9]:
### COLUMN_TRANSFORMER ###
##########################

# Preprocess the numerical features
numerical_processor = Pipeline(
    [
        (
            "num_scaler",
            MinMaxScaler(),
        )  # Shown in case it is needed. Not a must with decision trees.
    ]
)

# Preprocess the categorical features
# handle_unknown tells it to ignore (rather than throw an error for) any value
# that was not present in the initial training set.
categorical_processor = Pipeline(
    [("cat_encoder", OneHotEncoder(handle_unknown="ignore"))]
)

# Preprocess the text feature
text_processor_0 = Pipeline(
    [("text_vect_0", CountVectorizer(binary=True, max_features=150))]
)

# Combine all data preprocessors (add more if you choose to define more)
# For each processor/step, specify a name, the actual process, and the features to be processed
data_preprocessor = ColumnTransformer(
    [
        ("numerical_pre", numerical_processor, numerical_features),
        ("categorical_pre", categorical_processor, categorical_features),
        ("text_pre_0", text_processor_0, text_features[0]),
    ]
)

### PIPELINE ###
################

# Pipeline with all desired data transformers, along with an estimator
# Later, you can set/reach the parameters by using the names issued - for hyperparameter tuning, for example

##########################################################
### Notice the pipeline using a RandomForestClassifier ###
##########################################################
pipeline = Pipeline(
    [
        ("data_preprocessing", data_preprocessor),
        (
            "rf",
            RandomForestClassifier(
                max_depth=25, n_estimators=100  # Each tree has max_depth=25
            ),
        ),
    ]
)  # Use 100 trees

# Visualize the pipeline
# This will be helpful especially when building more complex pipelines,
# stringing together multiple preprocessing steps
from sklearn import set_config

set_config(display="diagram")
pipeline

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numerical_pre',
                                                  Pipeline(steps=[('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age upon Intake Days',
                                                   'Age upon Outcome Days']),
                                                 ('categorical_pre',
                                                  Pipeline(steps=[('cat_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex upon Outcome',
                                                   'Intake Type',
                                                   'Intake Condition',
                                                   'Pet Type',
                                                   'Sex upon Intake', 'Breed',
                                                   'Color']),
                                                 ('text_pre_0',
                                                  Pipeline(steps=[('text_vect_0',
                                                                   CountVectorizer(binary=True,
                                                                                   max_features=150))]),
                                                  'Found Location')])),
                ('rf', RandomForestClassifier(max_depth=25))])

Now you can fit the random forest model, and see the training and test scores.

In [10]:
# Get training data to train the pipeline
X_train = train_data[model_features]
y_train = train_data[model_target]

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Use the fitted pipeline to make predictions on the training dataset
train_predictions = pipeline.predict(X_train)
print(confusion_matrix(y_train, train_predictions))
print(classification_report(y_train, train_predictions))
print("Accuracy (training):", accuracy_score(y_train, train_predictions))

# Get testing data to test the pipeline
X_test = test_data[model_features]
y_test = test_data[model_target]

# Use the fitted pipeline to make predictions on the testing dataset
test_predictions = pipeline.predict(X_test)
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))
print("Accuracy (test):", accuracy_score(y_test, test_predictions))

[[25654 11845]
 [ 1769 46668]]
              precision    recall  f1-score   support

         0.0       0.94      0.68      0.79     37499
         1.0       0.80      0.96      0.87     48437

    accuracy                           0.84     85936
   macro avg       0.87      0.82      0.83     85936
weighted avg       0.86      0.84      0.84     85936

Accuracy (training): 0.8415797803016198
[[2770 1362]
 [ 219 5198]]
              precision    recall  f1-score   support

         0.0       0.93      0.67      0.78      4132
         1.0       0.79      0.96      0.87      5417

    accuracy                           0.83      9549
   macro avg       0.86      0.81      0.82      9549
weighted avg       0.85      0.83      0.83      9549

Accuracy (test): 0.8344329249136035


<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <h3><i>Try it yourself!</i></h3>
    <br>
    <p style="text-align:center;margin:auto;"><img src="images/challenge.png" alt="Challenge" width="100" /> </p>
    <p style=" text-align: center; margin: auto;">You can perform hyperparameter tuning on a random forest model.</p><br>
    <p style=" text-align: center; margin: auto;">In the following code cell, run a grid search with the random forest classifier using <code>param_grid={'rf__max_depth': [25, 30, 45]}</code>.</p><br>
    <p style=" text-align: center; margin: auto;">What is the best hyperparameter value after this run?</p>
    <br>
</div>

In [15]:
# Write your code for grid search with param_grid={'rf__max_depth': [25, 30, 45]}

# Parameter grid for GridSearch

############### CODE HERE ###############

from scipy.stats import randint
# Parameter grid for GridSearch

param_grid = {
    'rf__max_depth': [25, 30, 45]
}


grid_search = GridSearchCV(
    pipeline,  # Base model
    param_grid,  # Parameters to try
    cv=5,  # Apply 5-fold cross validation
    verbose=1,  # Print summaryGridSearchCV
    n_jobs=-1,  # Use all available processors
)

# Fit the RandomizedSearch to the training data
grid_search.fit(X_train, y_train)


############## END OF CODE ##############

print(grid_search.best_params_)
print(grid_search.best_score_)

# Get the best model out of GridSearchCV
classifier = grid_search.best_estimator_

# Fit the best model to the training data
classifier.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of

{'rf__max_depth': 45}
0.838542658634932


Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numerical_pre',
                                                  Pipeline(steps=[('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age upon Intake Days',
                                                   'Age upon Outcome Days']),
                                                 ('categorical_pre',
                                                  Pipeline(steps=[('cat_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex upon Outcome',
                                                   'Intake Type',
                                                   'Intake Condition',
                                                   'Pet Type',
                                                   'Sex upon Intake', 'Breed',
                                                   'Color']),
                                                 ('text_pre_0',
                                                  Pipeline(steps=[('text_vect_0',
                                                                   CountVectorizer(binary=True,
                                                                                   max_features=150))]),
                                                  'Found Location')])),
                ('rf', RandomForestClassifier(max_depth=45))])

In [16]:
# Get testing data to test the classifier
X_test = test_data[model_features]
y_test = test_data[model_target]

# Use the fitted model to make predictions on the test dataset
# Testing data going through the pipeline is first imputed
# (with means from the training set), scaled (with the min/max from the training data),
# and finally used to make predictions.
test_predictions = classifier.predict(X_test)

print("Model performance on the test set:")
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))
print("Test accuracy:", accuracy_score(y_test, test_predictions))

Model performance on the test set:
[[3012 1120]
 [ 356 5061]]
              precision    recall  f1-score   support

         0.0       0.89      0.73      0.80      4132
         1.0       0.82      0.93      0.87      5417

    accuracy                           0.85      9549
   macro avg       0.86      0.83      0.84      9549
weighted avg       0.85      0.85      0.84      9549

Test accuracy: 0.8454288407163054


---
<div class="alert alert-block alert-success">
<h1><span style="color:blue">Boosting</span></h1></div>

The last ensemble model that you will try is boosting. This method builds multiple weak models sequentially. Each subsequent model attempts to boost performance overall by overcoming or reducing the errors of the previous model.

You will use sklearn's [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) in the pipeline.

In [17]:
### COLUMN_TRANSFORMER ###
##########################

# Preprocess the numerical features
numerical_processor = Pipeline(
    [
        (
            "num_scaler",
            MinMaxScaler(),
        )  # Shown in case it is needed. Not a must with decision trees.
    ]
)

# Preprocess the categorical features
# handle_unknown tells it to ignore (rather than throw an error for) any value
# that was not present in the initial training set.
categorical_processor = Pipeline(
    [("cat_encoder", OneHotEncoder(handle_unknown="ignore"))]
)

# Preprocess the text feature
text_processor_0 = Pipeline(
    [("text_vect_0", CountVectorizer(binary=True, max_features=150))]
)

# Combine all data preprocessors (add more if you choose to define more)
# For each processor/step, specify a name, the actual process, and the features to be processed
data_preprocessor = ColumnTransformer(
    [
        ("numerical_pre", numerical_processor, numerical_features),
        ("categorical_pre", categorical_processor, categorical_features),
        ("text_pre_0", text_processor_0, text_features[0]),
    ]
)

### PIPELINE ###
################

# Pipeline with all desired data transformers, along with an estimator
# Later, you can set/reach the parameters by using the names issued - for hyperparameter tuning, for example

##############################################################
### Notice the pipeline using a GradientBoostingClassifier ###
##############################################################
pipeline = Pipeline(
    [
        ("data_preprocessing", data_preprocessor),
        (
            "gbc",
            GradientBoostingClassifier(
                max_depth=10, n_estimators=100  # Each tree has max_depth=10
            ),
        ),
    ]
)  # Use 100 trees

# Visualize the pipeline
# This will be helpful especially when building more complex pipelines,
# stringing together multiple preprocessing steps
from sklearn import set_config

set_config(display="diagram")
pipeline

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numerical_pre',
                                                  Pipeline(steps=[('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age upon Intake Days',
                                                   'Age upon Outcome Days']),
                                                 ('categorical_pre',
                                                  Pipeline(steps=[('cat_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex upon Outcome',
                                                   'Intake Type',
                                                   'Intake Condition',
                                                   'Pet Type',
                                                   'Sex upon Intake', 'Breed',
                                                   'Color']),
                                                 ('text_pre_0',
                                                  Pipeline(steps=[('text_vect_0',
                                                                   CountVectorizer(binary=True,
                                                                                   max_features=150))]),
                                                  'Found Location')])),
                ('gbc', GradientBoostingClassifier(max_depth=10))])

Now fit the model, and see the training and testing scores.

In [18]:
# Get training data to train the pipeline
X_train = train_data[model_features]
y_train = train_data[model_target]

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Use the fitted pipeline to make predictions on the training dataset
train_predictions = pipeline.predict(X_train)
print(confusion_matrix(y_train, train_predictions))
print(classification_report(y_train, train_predictions))
print("Accuracy (training):", accuracy_score(y_train, train_predictions))

# Get testing data to test the pipeline
X_test = test_data[model_features]
y_test = test_data[model_target]

# Use the fitted pipeline to make predictions on the testing dataset
test_predictions = pipeline.predict(X_test)
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))
print("Accuracy (test):", accuracy_score(y_test, test_predictions))

[[28076  9423]
 [ 2776 45661]]
              precision    recall  f1-score   support

         0.0       0.91      0.75      0.82     37499
         1.0       0.83      0.94      0.88     48437

    accuracy                           0.86     85936
   macro avg       0.87      0.85      0.85     85936
weighted avg       0.86      0.86      0.86     85936

Accuracy (training): 0.8580455222491156
[[3054 1078]
 [ 367 5050]]
              precision    recall  f1-score   support

         0.0       0.89      0.74      0.81      4132
         1.0       0.82      0.93      0.87      5417

    accuracy                           0.85      9549
   macro avg       0.86      0.84      0.84      9549
weighted avg       0.85      0.85      0.85      9549

Accuracy (test): 0.8486752539532936


---
<div class="alert alert-block alert-success">
<h1><span style="color:blue">Conclusion</span></h1></div>

This notebook provided an introduction to using Bagging, RandomForest, and GradientBoosting classifiers on the same dataset.

---
<div class="alert alert-block alert-success">
<h1><span style="color:blue">Next Lab</span></h1></div>

In the next lab, you will be introduced to fairness and bias mitigation in ML by exploring different types of bias that are present in data and practicing how to build various documentation sheets.